# Probing part 1

Vantage points will probe either the targets themselves (step 2) or other addresses in the same /24 prefix (step 2).

Vantage points are only the anchors.  
As always, targets are the anchors.  

This notebook is an implementation of the million scale method. Check the paper for more information.  
To do after create_datasets.ipynb

In [1]:
import uuid

from logger import logger
from scripts.utils.file_utils import load_json
from scripts.utils.measurement_utils import (
    load_targets, 
    load_vps,
    get_measurement_config,
    save_measurement_config,
    get_target_prefixes,
    ping_prefixes,
    ping_targets,
    retrieve_results,
    get_latest_measurements,
    insert_prefix_results,
    insert_target_results,
)
from default import (
    USER_ANCHORS_FILE, 
    USER_HITLIST_FILE, 
    PREFIX_MEASUREMENT_RESULTS,
    TARGET_MEASUREMENT_RESULTS,
    MEASUREMENT_CONFIG_PATH,
)

# will define the number of vps and targets to use
NB_TARGETS = 2
NB_VPS = 4

## Load targets and vps dataset

In [2]:

targets = load_targets(USER_ANCHORS_FILE, nb_target=NB_TARGETS)
vps = load_vps(USER_ANCHORS_FILE, nb_vps=NB_VPS)

# all target prefixes
target_prefixes = get_target_prefixes(targets)

# responsive IP addresses in all /24 prefixes
targets_per_prefix = load_json(USER_HITLIST_FILE)

logger.info(f"nb targets: {len(targets)}")
logger.info(f"nb_vps : {len(vps)}")

2023-09-20 22:49:46::INFO:root:2498612375:: nb targets: 2
2023-09-20 22:49:46::INFO:root:2498612375:: nb_vps : 4


## Generate measurement config

This configuration is used to retrieve all measurements results from RIPE Atlas using their API.

In [3]:
# measurement configuration for retrieval
experiment_uuid = uuid.uuid4()
target_measurement_uuid = uuid.uuid4()
prefix_measurement_uuid = uuid.uuid4()

logger.info(f"Starting experiment with uuid : {experiment_uuid}")

measurement_config = get_measurement_config(
    experiment_uuid=experiment_uuid,
    target_measurement_uuid=target_measurement_uuid,
    prefix_measurement_uuid=prefix_measurement_uuid,
    targets=targets, 
    target_prefixes=target_prefixes,
    vps=vps,
)

save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)

# Step 1: probing each target prefixes

## Probe target prefixes
WARNING : Time consumming section

In [4]:
ping_prefixes(
    measurement_uuid=prefix_measurement_uuid,
    measurement_config=measurement_config,
    target_prefixes=target_prefixes,
    targets_per_prefix=targets_per_prefix,
    vps=vps,
)

save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)

2023-09-20 22:49:46::INFO:root:measurement_utils:: Starting measurements 6fa82de2-507c-43eb-8b6f-80a5f6391f90 with parameters: dry_run=False; nb_targets=2; nb_vps=4.
2023-09-20 22:49:47::INFO:root:ping_and_traceroute_classes:: measurement tag: 6fa82de2-507c-43eb-8b6f-80a5f6391f90 : started measurement id : 60195879
2023-09-20 22:49:47::INFO:root:ping_and_traceroute_classes:: measurement tag: 6fa82de2-507c-43eb-8b6f-80a5f6391f90 : started measurement id : 60195880
2023-09-20 22:49:48::INFO:root:ping_and_traceroute_classes:: measurement tag: 6fa82de2-507c-43eb-8b6f-80a5f6391f90 : started measurement id : 60195881
2023-09-20 22:49:48::INFO:root:ping_and_traceroute_classes:: measurement tag: 6fa82de2-507c-43eb-8b6f-80a5f6391f90 : started measurement id : 60195882
2023-09-20 22:49:49::INFO:root:ping_and_traceroute_classes:: measurement tag: 6fa82de2-507c-43eb-8b6f-80a5f6391f90 : started measurement id : 60195883
2023-09-20 22:49:50::INFO:root:ping_and_traceroute_classes:: measurement tag: 6

# Step 2: probing each target

## Probe targets
WARNING : Time consumming section

In [5]:
# measurement configuration for retrieval
ping_targets(
    measurement_uuid=target_measurement_uuid,
    measurement_config=measurement_config,
    targets=targets,
    vps=vps,
    use_cache=False
)
save_measurement_config(measurement_config, MEASUREMENT_CONFIG_PATH)


2023-09-20 22:49:50::INFO:root:measurement_utils:: Starting measurements 47b8f0db-a9f2-4886-8b5c-0ad5ba667c9f with parameters: dry_run=False; nb_targets=2; nb_vps=4.
2023-09-20 22:49:50::INFO:root:ping_and_traceroute_classes:: measurement tag: 47b8f0db-a9f2-4886-8b5c-0ad5ba667c9f : started measurement id : 60195885
2023-09-20 22:49:51::INFO:root:ping_and_traceroute_classes:: measurement tag: 47b8f0db-a9f2-4886-8b5c-0ad5ba667c9f : started measurement id : 60195886
2023-09-20 22:49:51::INFO:root:ping_and_traceroute_classes:: measurement : 47b8f0db-a9f2-4886-8b5c-0ad5ba667c9f done


## Retrieve prefix results
WARNING : Time consuming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [6]:
measurement_config = get_latest_measurements(MEASUREMENT_CONFIG_PATH)
logger.info(measurement_config)

for measurement_id in measurement_config["target_measurements"]["ids"]:
    print(measurement_id)

"https://atlas.ripe.net/api/v2/measurements/60195539/"


2023-09-20 22:49:51::INFO:root:2416656981:: {'experiment_uuid': 'e05b7508-bb3f-406e-a839-e2178687425a', 'status': 'finished', 'start_time': '2023-09-20 22:49:46.595547', 'end_time': '2023-09-20 22:49:51.081318', 'is_dry_run': False, 'nb_targets': 2, 'nb_vps': 4, 'description': 'measurements from a set of vps towards all targets/target prefixes', 'af': 4, 'target_measurements': {'measurement_uuid': '47b8f0db-a9f2-4886-8b5c-0ad5ba667c9f', 'targets': ['185.159.159.9', '200.132.1.32'], 'vps': {'45.129.95.8': {'id': 7132, 'address_v4': '45.129.95.8', 'asn_v4': 207960, 'country_code': 'GB', 'geometry': {'type': 'Point', 'coordinates': [-3.1615, 51.4985]}}, '193.201.40.210': {'id': 6235, 'address_v4': '193.201.40.210', 'asn_v4': 24796, 'country_code': 'IT', 'geometry': {'type': 'Point', 'coordinates': [12.5085, 41.8995]}}, '93.90.135.201': {'id': 7096, 'address_v4': '93.90.135.201', 'asn_v4': 47295, 'country_code': 'DE', 'geometry': {'type': 'Point', 'coordinates': [14.3515, 51.1905]}}, '103.

60195885
60195886


'https://atlas.ripe.net/api/v2/measurements/60195539/'

In [1]:
import uuid

from logger import logger
from scripts.utils.file_utils import load_json
from scripts.utils.measurement_utils import (
    load_targets, 
    load_vps,
    get_measurement_config,
    save_measurement_config,
    get_target_prefixes,
    ping_prefixes,
    ping_targets,
    retrieve_results,
    get_latest_measurements,
    insert_prefix_results,
    insert_target_results,
)
from default import (
    USER_ANCHORS_FILE, 
    USER_HITLIST_FILE, 
    PREFIX_MEASUREMENT_RESULTS,
    TARGET_MEASUREMENT_RESULTS,
    MEASUREMENT_CONFIG_PATH,
)

# will define the number of vps and targets to use
NB_TARGETS = 2
NB_VPS = 4

In [5]:
measurement_config = get_latest_measurements(MEASUREMENT_CONFIG_PATH)
logger.info(measurement_config)

prefix_measurement_uuid = measurement_config["prefix_measurements"]['measurement_uuid']

logger.info(f"retrieving results for measurement ids: {prefix_measurement_uuid}")

# sometimes, not all probes give output, reduce timeout if you do not want to wait for too long
response = retrieve_results(prefix_measurement_uuid,PREFIX_MEASUREMENT_RESULTS, NB_TARGETS, NB_VPS, timeout= 5)

insert_prefix_results(response)

2023-09-21 09:40:52::INFO:root:1913282727:: {'experiment_uuid': 'e05b7508-bb3f-406e-a839-e2178687425a', 'status': 'finished', 'start_time': '2023-09-20 22:49:46.595547', 'end_time': '2023-09-20 22:49:51.081318', 'is_dry_run': False, 'nb_targets': 2, 'nb_vps': 4, 'description': 'measurements from a set of vps towards all targets/target prefixes', 'af': 4, 'target_measurements': {'measurement_uuid': '47b8f0db-a9f2-4886-8b5c-0ad5ba667c9f', 'targets': ['185.159.159.9', '200.132.1.32'], 'vps': {'45.129.95.8': {'id': 7132, 'address_v4': '45.129.95.8', 'asn_v4': 207960, 'country_code': 'GB', 'geometry': {'type': 'Point', 'coordinates': [-3.1615, 51.4985]}}, '193.201.40.210': {'id': 6235, 'address_v4': '193.201.40.210', 'asn_v4': 24796, 'country_code': 'IT', 'geometry': {'type': 'Point', 'coordinates': [12.5085, 41.8995]}}, '93.90.135.201': {'id': 7096, 'address_v4': '93.90.135.201', 'asn_v4': 47295, 'country_code': 'DE', 'geometry': {'type': 'Point', 'coordinates': [14.3515, 51.1905]}}, '103.

NetworkError: Code: 210. Connection refused (localhost:9000)

## Retrieve traget results
WARNING : Time consumming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [ ]:
measurement_config = get_latest_measurements(MEASUREMENT_CONFIG_PATH)

target_measurement_uuid = measurement_config["target_measurements"]["measurement_uuid"]

response = retrieve_results(target_measurement_uuid, TARGET_MEASUREMENT_RESULTS, NB_TARGETS, NB_VPS)

insert_target_results(response)

2023-09-13 15:41:07::INFO:root:measurement_utils:: nb measurements retrieved: 8
2023-09-13 15:41:07::INFO:root:measurement_utils:: Target measurements successfully inserted in table : user_vps_to_target
